In [1]:
# --- Imports ---
import pandas as pd
import numpy as np
import os
from surprise import Dataset, Reader, SVD, dump
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

In [2]:
# --- Setup paths ---
current_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

file_path = os.path.join(current_dir, 'data', 'processed', 'hotelrec_2013_2017_cleaned_encoded.csv.gz')
cluster_path = os.path.join(current_dir, 'data', 'processed', 'hotel_clusters.csv')
model_save_path = os.path.join(current_dir, 'models')
os.makedirs(model_save_path, exist_ok=True)

In [3]:
# --- Load hotel clusters ---
hotel_clusters = pd.read_csv(cluster_path)

In [ ]:
# --- Initialize storage for meta-model stacking --- [~12 min]
all_pred_svd = []
all_pred_sentiment = []
all_pred_hybrid = []
all_true = []

chunk_size = 500_000  # safer for memory

for chunk_idx, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
    print(f"\nProcessing Chunk {chunk_idx+1}...")

    # --- Merge hotel clusters ---
    chunk = chunk.merge(hotel_clusters[['hotel_name_id', 'hotel_cluster']], on='hotel_name_id', how='left')

    # --- Fill missing values ---
    structured_features = ['sleep quality', 'value', 'rooms', 'service', 'cleanliness', 'location']
    chunk[structured_features] = chunk[structured_features].fillna(chunk[structured_features].mean())

    # --- Prepare LightGBM Features ---
    feature_cols = ['hotel_name_id', 'author_id', 'sentiment_score'] + structured_features + ['hotel_cluster']
    X = chunk[feature_cols]
    y = chunk['rating']

    # --- Prepare SVD Data (separately) ---
    svd_df = chunk[['author', 'hotel_id', 'rating']]

    # --- Train-Test Split for LightGBM ---
    X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X, y, test_size=0.2, random_state=42)

    # --- Train-Test Split for SVD ---
    svd_train_df, svd_test_df = train_test_split(svd_df, test_size=0.2, random_state=42)

    # --- Train Sentiment-Aware LightGBM ---
    X_train_sentiment = X_train_full[['hotel_name_id', 'author_id', 'sentiment_score']]
    X_test_sentiment = X_test_full[['hotel_name_id', 'author_id', 'sentiment_score']]

    model_sentiment = lgb.LGBMRegressor(
        objective='regression',
        n_estimators=500,
        learning_rate=0.05,
        num_leaves=31,
        random_state=42
    )
    model_sentiment.fit(
        X_train_sentiment, y_train_full,
        eval_set=[(X_test_sentiment, y_test_full)],
        callbacks=[early_stopping(stopping_rounds=20), log_evaluation(50)]
    )

    preds_sentiment = model_sentiment.predict(X_test_sentiment)

    # --- Train Hybrid LightGBM (sentiment + subratings) ---
    model_hybrid = lgb.LGBMRegressor(
        objective='regression',
        n_estimators=500,
        learning_rate=0.05,
        num_leaves=31,
        random_state=42
    )
    model_hybrid.fit(
        X_train_full, y_train_full,
        eval_set=[(X_test_full, y_test_full)],
        callbacks=[early_stopping(stopping_rounds=20), log_evaluation(50)]
    )

    preds_hybrid = model_hybrid.predict(X_test_full)

    # --- Train SVD Model ---
    reader = Reader(rating_scale=(1, 5))
    surprise_data = Dataset.load_from_df(svd_train_df[['author', 'hotel_id', 'rating']], reader)
    trainset = surprise_data.build_full_trainset()

    model_svd = SVD(n_factors=150, n_epochs=20, lr_all=0.005, reg_all=0.02)
    model_svd.fit(trainset)

    # --- Predict batch for SVD ---
    def svd_predict_batch(model, df_batch):
        preds = []
        for idx, row in tqdm(df_batch.iterrows(), total=len(df_batch), desc="SVD Predicting"):
            preds.append(model.predict(row['author'], row['hotel_id']).est)
        return np.array(preds)

    preds_svd = svd_predict_batch(model_svd, svd_test_df)

    # --- Store predictions ---
    all_pred_svd.append(preds_svd)
    all_pred_sentiment.append(preds_sentiment)
    all_pred_hybrid.append(preds_hybrid)
    all_true.append(y_test_full.values)

    print(f"Chunk {chunk_idx+1} done.")


Processing Chunk 1...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001768 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 3
[LightGBM] [Info] Start training from score 4.208140
Training until validation scores don't improve for 20 rounds
[50]	valid_0's l2: 0.55034
[100]	valid_0's l2: 0.539952
[150]	valid_0's l2: 0.536647
[200]	valid_0's l2: 0.535035
[250]	valid_0's l2: 0.534251
[300]	valid_0's l2: 0.534021
[350]	valid_0's l2: 0.533707
[400]	valid_0's l2: 0.533453
[450]	valid_0's l2: 0.533295
Early stopping, best iteration is:
[459]	valid_0's l2: 0.533213
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003952 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if mem

SVD Predicting: 100%|██████████| 100000/100000 [00:02<00:00, 40535.54it/s]


Chunk 1 done.

Processing Chunk 2...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000501 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 3
[LightGBM] [Info] Start training from score 4.158680
Training until validation scores don't improve for 20 rounds
[50]	valid_0's l2: 0.583786
[100]	valid_0's l2: 0.573056
[150]	valid_0's l2: 0.570345
[200]	valid_0's l2: 0.569462
[250]	valid_0's l2: 0.568887
[300]	valid_0's l2: 0.568518
[350]	valid_0's l2: 0.568354
[400]	valid_0's l2: 0.568158
Early stopping, best iteration is:
[416]	valid_0's l2: 0.568113
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002555 seconds.
You can set `force

SVD Predicting: 100%|██████████| 100000/100000 [00:02<00:00, 38982.39it/s]


Chunk 2 done.

Processing Chunk 3...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000760 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 3
[LightGBM] [Info] Start training from score 4.148167
Training until validation scores don't improve for 20 rounds
[50]	valid_0's l2: 0.578446
[100]	valid_0's l2: 0.566602
[150]	valid_0's l2: 0.564088
[200]	valid_0's l2: 0.563158
[250]	valid_0's l2: 0.562689
[300]	valid_0's l2: 0.562353
[350]	valid_0's l2: 0.562195
Early stopping, best iteration is:
[362]	valid_0's l2: 0.562117
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003654 seconds.
You can set `force_row_wise=true` to remove the

...

Chunk 64 done.

Processing Chunk 65...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000591 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 3
[LightGBM] [Info] Start training from score 4.149095
Training until validation scores don't improve for 20 rounds
[50]	valid_0's l2: 0.585484
[100]	valid_0's l2: 0.575502
[150]	valid_0's l2: 0.572994
[200]	valid_0's l2: 0.57231
[250]	valid_0's l2: 0.571828
[300]	valid_0's l2: 0.571492
[350]	valid_0's l2: 0.571288
[400]	valid_0's l2: 0.57108
[450]	valid_0's l2: 0.571001
[500]	valid_0's l2: 0.570946
Did not meet early stopping. Best iteration is:
[497]	valid_0's l2: 0.570941
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-thre

SVD Predicting: 100%|██████████| 100000/100000 [00:02<00:00, 39434.11it/s]


Chunk 65 done.

Processing Chunk 66...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000336 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 314009, number of used features: 3
[LightGBM] [Info] Start training from score 4.126837
Training until validation scores don't improve for 20 rounds
[50]	valid_0's l2: 0.585086
[100]	valid_0's l2: 0.573155
[150]	valid_0's l2: 0.570195
[200]	valid_0's l2: 0.568801
[250]	valid_0's l2: 0.56816
[300]	valid_0's l2: 0.567616
[350]	valid_0's l2: 0.567302
[400]	valid_0's l2: 0.567097
[450]	valid_0's l2: 0.567031
Early stopping, best iteration is:
[464]	valid_0's l2: 0.566986
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.0018

SVD Predicting: 100%|██████████| 78503/78503 [00:02<00:00, 38987.68it/s]

Chunk 66 done.


In [6]:
# --- Stack all predictions together ---
pred_svd_all = np.concatenate(all_pred_svd)
pred_sentiment_all = np.concatenate(all_pred_sentiment)
pred_hybrid_all = np.concatenate(all_pred_hybrid)
y_true_all = np.concatenate(all_true)

In [7]:
stacked_X = np.vstack([pred_svd_all, pred_sentiment_all, pred_hybrid_all]).T
stacked_y = y_true_all

In [8]:
# --- Final Train-Test Split for Meta-Model ---
X_train_stack, X_test_stack, y_train_stack, y_test_stack = train_test_split(stacked_X, stacked_y, test_size=0.2, random_state=42)

In [9]:
# --- Train Meta-Model (Linear Regression) ---
meta_model = LinearRegression()
meta_model.fit(X_train_stack, y_train_stack)

LinearRegression()

In [10]:
# --- Predict and Evaluate ---
y_pred_stack = meta_model.predict(X_test_stack)

rmse = mean_squared_error(y_test_stack, y_pred_stack, squared=False)
mae = mean_absolute_error(y_test_stack, y_pred_stack)

In [11]:
print("\n=== Final Stacked Ensemble Model Results ===")
print(f"RMSE: {rmse:.4f}")
print(f"MAE:  {mae:.4f}")


=== Final Stacked Ensemble Model Results ===
RMSE: 0.5418
MAE:  0.3754


In [14]:
# --- Save Meta-Model ---
joblib.dump(meta_model, os.path.join(model_save_path, 'meta_model.pkl'))

print("Saved final stacked meta-model.")

Saved final stacked meta-model.
